In [5]:
import pandas as pd
import os
import requests
import math
from datetime import datetime

In [49]:
# Загружем последний сохраненный файл с обработанными данными (= количеству уникальных организаций)
output_file_1 = r"C:\Users\Mi\Documents\Защита проекта\financial_locations\data\processed3\svod_df_processed_20250809_1959.csv"
# Загрузка данных
loaded_df = pd.read_csv(
    output_file_1,
    encoding='utf-8-sig',
    parse_dates=['dt', 'cdt', 'dtvkl', 'mds', 'mde'],  # Автоматическое преобразование дат
    dtype={
        'inn': str,             # Сохранить как строку (чтобы не потерять нули)
        'ogrn': str
    },
    na_values=['NA', 'N/A']     # Какие значения считать пропущенными
)
# Проверка загрузки
print(f"Загружено строк: {len(loaded_df)}")
print(loaded_df.head())

Загружено строк: 6968
           dt                 cdt                   name                  lic  \
0  2025-01-01 2025-01-09 10:05:57          ООО «МКК АДК»  00-16-035-32-007640   
1  2025-01-01 2025-01-09 10:05:57  ООО "ЛОМБАРД-КАРАТ +"                  NaN   
2  2025-01-01 2025-01-09 10:05:57           КПКГ"ДРУЖБА"                  NaN   
3  2025-01-01 2025-01-09 10:05:57        СКПК "ПРОГРЕСС"                  NaN   
4  2025-01-01 2025-01-09 10:05:57         ООО «Лидер УК»                  579   

          inn           ogrn                                         adr  \
0  4230030712  1164205052428           Юрга, пр-т Победы, д. 18, кв. 203   
1  4230033223  1194205010361              , ПР-КТ ПОБЕДЫ, Д. 41А, ОФИС 2   
2  4240005761  1024202202441        ПРОМЫШЛЕННАЯ, ПЕР. ТЕАТРАЛЬНЫЙ, Д. 3   
3  4240008522  1064240003497  ПРОМЫШЛЕННАЯ, УЛ. КООПЕРАТИВНАЯ, Д.4, -, -   
4  4246019520  1144246000700      Кемерово, ул. Бийская, д. 38, офис 204   

        dtvkl         mds         

In [51]:
# Загружем первоначальный файл со всеми организациями на 01.01.2025 
output_file_2 = r"C:\Users\Mi\Documents\Защита проекта\financial_locations\data\processed3\dbrd.csv"
# Загрузка данных
loaded_df_2 = pd.read_csv(
    output_file_2,
    encoding='utf-8-sig',
    parse_dates=['dt', 'cdt', 'dtvkl', 'mds', 'mde'],  # Автоматическое преобразование дат
    dtype={
        'inn': str,             # Сохранить как строку (чтобы не потерять нули)
        'ogrn': str
    },
    na_values=['NA', 'N/A']     # Какие значения считать пропущенными
)
# Проверка загрузки
print(f"Загружено строк: {len(loaded_df_2)}")
print(loaded_df_2.head())

Загружено строк: 8753
          dt                 cdt                   name                  lic  \
0 2025-01-01 2025-01-09 10:05:57          ООО «МКК АДК»  00-16-035-32-007640   
1 2025-01-01 2025-01-09 10:05:57  ООО "ЛОМБАРД-КАРАТ +"                  NaN   
2 2025-01-01 2025-01-09 10:05:57           КПКГ"ДРУЖБА"                  NaN   
3 2025-01-01 2025-01-09 10:05:57        СКПК "ПРОГРЕСС"                  NaN   
4 2025-01-01 2025-01-09 10:05:57         ООО «Лидер УК»                  579   

          inn           ogrn                                         adr  \
0  4230030712  1164205052428           Юрга, пр-т Победы, д. 18, кв. 203   
1  4230033223  1194205010361              , ПР-КТ ПОБЕДЫ, Д. 41А, ОФИС 2   
2  4240005761  1024202202441        ПРОМЫШЛЕННАЯ, ПЕР. ТЕАТРАЛЬНЫЙ, Д. 3   
3  4240008522  1064240003497  ПРОМЫШЛЕННАЯ, УЛ. КООПЕРАТИВНАЯ, Д.4, -, -   
4  4246019520  1144246000700      Кемерово, ул. Бийская, д. 38, офис 204   

       dtvkl        mds        mde  ... 

In [29]:
print(loaded_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6968 entries, 0 to 6967
Data columns (total 40 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   dt                6968 non-null   object        
 1   cdt               6968 non-null   datetime64[ns]
 2   name              6958 non-null   object        
 3   lic               2838 non-null   object        
 4   inn               6967 non-null   object        
 5   ogrn              6869 non-null   object        
 6   adr               6884 non-null   object        
 7   dtvkl             6968 non-null   object        
 8   mds               6968 non-null   object        
 9   mde               6968 non-null   object        
 10  numfo             6968 non-null   int64         
 11  fo                6968 non-null   object        
 12  srf               6968 non-null   object        
 13  numsrf            6968 non-null   int64         
 14  okato             6968 n

In [53]:
# 1. Создаем список столбцов для копирования из loaded_df в loaded_df_2
cols_to_copy = [
    'merge_key_x', 'source_address', 'source_inn', 'source_ogrn',
    'found_by', 'result', 'postal_code', 'federal_district', 'region',
    'area', 'city', 'street', 'house', 'geo_lat', 'geo_lon', 'source',
    'merge_key_y'
]

# 2. Создаем копию исходного датафрейма
final_df = loaded_df_2.copy()

# 3. Добавляем все недостающие столбцы в final_df
for col in cols_to_copy:
    if col not in final_df.columns:
        final_df[col] = None  # Инициализируем пустыми значениями

In [57]:
# 4. Получаем уникальные INN из обоих датафреймов
unique_inn_loaded_df = set(loaded_df['inn'].dropna().unique())
unique_inn_loaded_df_2 = set(loaded_df_2['inn'].dropna().unique())

# 5. Проверяем покрытие
missing_inn = unique_inn_loaded_df_2 - unique_inn_loaded_df
if missing_inn:
    print(f"Предупреждение: {len(missing_inn)} INN из loaded_df_2 отсутствуют в loaded_df")

Предупреждение: 239 INN из loaded_df_2 отсутствуют в loaded_df


In [67]:
# Находим INN, которые есть в loaded_df_2, но отсутствуют в loaded_df
missing_inn = list(unique_inn_loaded_df_2 - unique_inn_loaded_df)

# Создаем таблицу сравнения
comparison_table = loaded_df_2[loaded_df_2['inn'].isin(missing_inn)][['inn', 'ogrn', 'name', 'full_adr']].copy()

# Добавляем пометку об отсутствии
comparison_table['status'] = 'Отсутствует в loaded_df'

# Добавляем соответствующие данные из loaded_df (если есть частичные совпадения)
if len(missing_inn) > 0:
    print("\n" + "="*80)
    print(f"Детализация по {len(missing_inn)} отсутствующим INN:")
    print("="*80)
    
    # Выводим таблицу с важными столбцами
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 1000)
    pd.set_option('display.max_colwidth', 40)
    
    print(comparison_table[['inn', 'name', 'full_adr', 'status']].sort_values('inn'))
    
    # Сохраняем в отдельный файл для детального анализа
    output_dir = r"C:\Users\Mi\Documents\Защита проекта\financial_locations\data\processed3"
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    missing_inn_path = os.path.join(output_dir, f"missing_inn_{timestamp}.csv")
    comparison_table.to_csv(missing_inn_path, index=False, encoding='utf-8-sig')
    print(f"\nПолный список отсутствующих INN сохранен в: {missing_inn_path}")
else:
    print("\nВсе INN из loaded_df_2 присутствуют в loaded_df - расхождений нет")


Детализация по 239 отсутствующим INN:
               inn                                     name                                 full_adr                   status
143     0100006246                    ООО "ЛОМБАРД "КАРМАН"   ЮЖНЫЙ ФЕДЕРАЛЬНЫЙ ОКРУГ, Республика...  Отсутствует в loaded_df
144   010106751003                             Худолей А.В.   ЮЖНЫЙ ФЕДЕРАЛЬНЫЙ ОКРУГ, Республика...  Отсутствует в loaded_df
24      0102000578                   ООО "Вайлдберриз Банк"   ЦЕНТРАЛЬНЫЙ ФЕДЕРАЛЬНЫЙ ОКРУГ, г. М...  Отсутствует в loaded_df
146     0102000578                   ООО "Вайлдберриз Банк"   ЦЕНТРАЛЬНЫЙ ФЕДЕРАЛЬНЫЙ ОКРУГ, г. М...  Отсутствует в loaded_df
145     0102000578                   ООО "Вайлдберриз Банк"   ЦЕНТРАЛЬНЫЙ ФЕДЕРАЛЬНЫЙ ОКРУГ, г. М...  Отсутствует в loaded_df
130     0103001895                         ООО КБ "ГТ банк"   ЮЖНЫЙ ФЕДЕРАЛЬНЫЙ ОКРУГ, Краснодарс...  Отсутствует в loaded_df
129     0103001895                         ООО КБ "ГТ банк"   ЮЖНЫЙ ФЕДЕРАЛЬНЫЙ

In [65]:
# Дополнительная проверка: организации, которые есть в loaded_df, но нет в loaded_df_2
extra_inn = list(unique_inn_loaded_df - unique_inn_loaded_df_2)
if extra_inn:
    print("\n" + "="*80)
    print(f"Обнаружено {len(extra_inn)} INN в loaded_df, которых нет в loaded_df_2:")
    print("="*80)
    extra_table = loaded_df[loaded_df['inn'].isin(extra_inn)][['inn', 'ogrn', 'name', 'full_adr']]
    extra_table['status'] = 'Отсутствует в loaded_df_2'
    print(extra_table[['inn', 'name', 'full_adr', 'status']].sort_values('inn'))


Обнаружено 239 INN в loaded_df, которых нет в loaded_df_2:
              inn                                     name                                 full_adr                     status
124     100006246                    ООО "ЛОМБАРД "КАРМАН"   ЮЖНЫЙ ФЕДЕРАЛЬНЫЙ ОКРУГ, Республика...  Отсутствует в loaded_df_2
125   10106751003                             Худолей А.В.   ЮЖНЫЙ ФЕДЕРАЛЬНЫЙ ОКРУГ, Республика...  Отсутствует в loaded_df_2
19      102000578                   ООО "Вайлдберриз Банк"   ЦЕНТРАЛЬНЫЙ ФЕДЕРАЛЬНЫЙ ОКРУГ, г. М...  Отсутствует в loaded_df_2
20      103001895                         ООО КБ "ГТ банк"   ЮЖНЫЙ ФЕДЕРАЛЬНЫЙ ОКРУГ, Краснодарс...  Отсутствует в loaded_df_2
126     104013117                        КПК "АГРОРОСТОРГ"   ЮЖНЫЙ ФЕДЕРАЛЬНЫЙ ОКРУГ, Краснодарс...  Отсутствует в loaded_df_2
112     104013276                    ООО "Ломбард Лазурит"   ЮЖНЫЙ ФЕДЕРАЛЬНЫЙ ОКРУГ, Республика...  Отсутствует в loaded_df_2
127     104013283                           КПК "СВ

In [69]:
# 1. Находим INN, которые есть в loaded_df_2, но отсутствуют в loaded_df
initial_missing = list(set(loaded_df_2['inn'].dropna()) - set(loaded_df['inn'].dropna()))

# 2. Проверяем, какие из них могут совпадать, если добавить ведущий ноль в loaded_df
potential_matches = []
for inn in initial_missing:
    # Убираем ведущий ноль в INN из loaded_df_2 для поиска соответствий
    short_inn = inn.lstrip('0')
    if short_inn in loaded_df['inn'].values:
        potential_matches.append((inn, short_inn))

# 3. Обновляем INN в loaded_df для этих случаев
for full_inn, short_inn in potential_matches:
    mask = loaded_df['inn'] == short_inn
    loaded_df.loc[mask, 'inn'] = full_inn  # Заменяем на версию с ведущим нулем

# 4. Проверяем остаточные расхождения после исправления
updated_missing = list(set(loaded_df_2['inn'].dropna()) - set(loaded_df['inn'].dropna()))
print(f"Осталось несоответствий после исправления: {len(updated_missing)}")


Осталось несоответствий после исправления: 0


In [71]:
# 5. Выводим таблицу с оставшимися расхождениями (если есть)
if updated_missing:
    mismatch_table = loaded_df_2[loaded_df_2['inn'].isin(updated_missing)][['inn', 'name', 'full_adr']]
    print("\nОставшиеся несоответствия:")
    print(mismatch_table.head(10))  # Показываем первые 10 для примера
else:
    print("\nВсе INN теперь имеют соответствия!")


Все INN теперь имеют соответствия!


In [79]:
# Подготавливаем данные для объединения (убираем дубликаты)
merge_df = loaded_df[['inn', 'ogrn'] + cols_to_copy].drop_duplicates(subset=['inn', 'ogrn'])

In [81]:
# Объединяем данные
final_df = final_df.merge(
    merge_df,
    on=['inn', 'ogrn'],
    how='left',
    suffixes=('_original', '_from_loaded')
)

In [83]:
# Обновляем значения в результирующем датафрейме
for col in cols_to_copy:
    # Обновляем только если значение было пустым
    final_df[col] = final_df[col].mask(
        final_df[col].isna(),
        final_df[f'{col}_from_loaded']
    )
    # Удаляем временный столбец
    final_df.drop(f'{col}_from_loaded', axis=1, inplace=True)

In [87]:
# 7. Подсчет результатов
# Находим строки, где хотя бы один целевой столбец был обновлен
update_mask = final_df[cols_to_copy].notna().any(axis=1)
updated_rows = update_mask.sum()

# Уникальные INN с обновлениями
updated_unique_inn = final_df.loc[update_mask, 'inn'].nunique()

In [89]:
print(f"\nИтоговые результаты:")
print(f"Общее количество обновленных строк: {updated_rows}")
print(f"Количество уникальных обновленных INN: {updated_unique_inn}")


Итоговые результаты:
Общее количество обновленных строк: 8753
Количество уникальных обновленных INN: 6967


In [91]:
# 8. Дополнительная обработка перед сохранением
# Конвертируем INN и OGRN в строковый тип 
final_df['inn'] = final_df['inn'].astype(str)
final_df['ogrn'] = final_df['ogrn'].astype(str)

# Удаляем возможные '.0' у числовых значений, которые стали строками
for col in ['inn', 'ogrn']:
    final_df[col] = final_df[col].str.replace(r'\.0$', '', regex=True)

# 11. Сохранение в CSV 
output_dir = r"C:\Users\Mi\Documents\Защита проекта\financial_locations\data\processed3"
os.makedirs(output_dir, exist_ok=True)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

In [95]:
# CSV сохранение
csv_output_path = os.path.join(output_dir, f"Обработанные_исходные_данные_{timestamp}.csv")
final_df.to_csv(
    csv_output_path,
    index=False,
    encoding='utf-8-sig',
    sep=';',
    decimal=',',
    date_format='%Y-%m-%d',
    
)

In [105]:
# 8. Дополнительная обработка перед сохранением
import csv
# Конвертируем INN и OGRN в строковый тип и добавляем кавычки
final_df['inn'] = "'" + final_df['inn'].astype(str).str.replace(r'\.0$', '', regex=True)
final_df['ogrn'] = "'" + final_df['ogrn'].astype(str).str.replace(r'\.0$', '', regex=True)

# 9. Сохранение в CSV с защитой формата
output_dir = r"C:\Users\Mi\Documents\Защита проекта\financial_locations\data\processed3"
os.makedirs(output_dir, exist_ok=True)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
csv_output_path = os.path.join(output_dir, f"Обработанные_исходные_данные_{timestamp}.csv")

# Сохраняем с дополнительными параметрами для защиты формата
final_df.to_csv(
    csv_output_path,
    index=False,
    encoding='utf-8-sig',
    sep=';',
    decimal=',',
    date_format='%Y-%m-%d',
    quoting=csv.QUOTE_ALL  # Все значения в кавычках
)

print(f"Файл успешно сохранен: {csv_output_path}")


Файл успешно сохранен: C:\Users\Mi\Documents\Защита проекта\financial_locations\data\processed3\Обработанные_исходные_данные_20250810_142457.csv
